In [2]:
import autorootcwd
import chromadb
import requests
import os
import pandas as pd
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from src.exploratory_data_analysis import gather_data
from sentence_transformers import SentenceTransformer
import uuid

In [27]:
articles = gather_data(path='data/2023_articles_en/')

In [31]:
articles = articles[~articles['categories'].str.contains('AD|SE|RU', case=False, na=False)]

In [32]:
articles

,byline,channels,desk,headline,keywords,categories,lede,places,priority,id,...,previous,bert_embedding,label,high_label,mid_label,ada_embedding,date,correction,editorialRemarks,videoAlbums
3,sm/sag/sm,['STA'],AN,Government and unions resume wage reform talks,"['PUBLIC SECTOR', 'WAGES', 'GOVERNMENT', 'TRAD...",['PO'],"A month after their last meeting, the governme...","[{'city': 'Ljubljana', 'country': 'SLOVENIA', ...",3,3154199,...,NaN,"[-0.181034327, -0.0897886902, 0.400760919, -0....",teachers union,education,teachers union,"[-0.01332863699644804, -0.009512553922832012, ...",2023-03-27,NaN,NaN,NaN
4,sm/sm,['STA'],AN,Finance pessimistic about EU's green mobility ...,"['PRESS', 'AUTOMOTIVE', 'RENEWABLES', 'EU']",['BE'],Looking at Europe's current policies regarding...,"[{'city': 'Ljubljana', 'country': 'SLOVENIA', ...",4,3154197,...,NaN,"[-0.149146557, -0.0423529632, 0.584996104, 0.1...",european elections,environmental issue,motor rallying,"[0.016671577468514442, -0.01644466072320938, 0...",2023-03-27,NaN,NaN,NaN
5,sm/sm,['STA'],AN,Weather: Mini cold snap,['WEATHER'],['AS'],An incursion of cold air from the north will d...,"[{'city': 'Ljubljana', 'country': 'SLOVENIA', ...",4,3154056,...,NaN,"[-0.422773331, -0.151280552, 0.656639576, 0.00...",weather,weather,weather science,"[0.0005583065212704241, -0.011048422195017338,...",2023-03-27,NaN,NaN,NaN
6,sys/mab,['STA'],AN,"Daily headlines - Monday, 27 March","['PRESS', 'REVIEW']",['PO'],Below is a review of the headlines in Slovenia...,"[{'city': 'Ljubljana', 'country': 'SLOVENIA', ...",4,3154241,...,NaN,"[-0.106269322, 0.0715350211, 0.346314758, -0.2...",labor market,"economy, business and finance",tourism and leisure,"[-0.006873154081404209, 0.00819583423435688, 0...",2023-03-27,NaN,NaN,NaN
7,mab/ep,['STA'],AN,Dnevnik says administrative units product of s...,"['PRESS', 'ADMINISTRATION', 'OPPOSITION']",['PO'],The newspaper Dnevnik comments on Monday on th...,"[{'city': 'Ljubljana', 'country': 'SLOVENIA', ...",4,3154266,...,NaN,"[-0.226806596, -0.00745235709, 0.308664531, -0...",government departments,politics,local authority,"[0.008214174769818783, 0.006234936881810427, 0...",2023-03-27,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9278,gz/sm,['STA'],AN,Rights groups urge Koper port to implement out...,"['TRANSPORT', 'LUKA KOPER', 'LABOUR', 'RIGHTS'...","['BE', 'PO']",Labour rights groups urged port operator Luka ...,"[{'city': 'Koper', 'country': 'SLOVENIA', 'cod...",4,3124497,...,NaN,"[-0.10780827, -0.00200381526, 0.212821946, -0....",labour dispute,labour,labour dispute,"[0.013940882869064808, -0.012297076173126698, ...",2023-01-05,NaN,NaN,NaN
9279,sm/ep,['STA'],AN,Slovenian IoT provider GOAP acquired by Allterco,"['COMPANIES', 'TECHNOLOGY', 'TAKEOVER']",['BE'],"Allterco, a Bulgarian developer of internet of...","[{'city': 'Sofia', 'country': 'BULGARIA', 'cod...",4,3124524,...,NaN,"[-0.377420098, -0.14120689, 0.602269232, 0.017...",Grand Prix,sport,company information,"[0.0036020863335579634, -0.01850098930299282, ...",2023-01-05,NaN,NaN,NaN
9280,eho/gm/sm,['STA'],AN,Slovenian co-operative allegedly victim of fraud,"['AGRICULTURE', 'FRANCE']",['BE'],Slovenian police are investigating suspicion t...,"[{'city': 'Slovenj Gradec', 'country': 'SLOVEN...",4,3124526,...,NaN,"[-0.246493384, -0.123667672, 0.282776266, -0.2...",forestry and timber,"crime, law and justice",corporate crime,"[0.011181429959833622, -0.02400280348956585, 0...",2023-01-05,NaN,NaN,NaN
9281,mas/sm,['STA'],AN,Medical wholesaler Salus buys another rival,"['COMPANIES', 'SALE', 'PHARMACEUTICALS']","['BE', 'HE']",Pharmaceutical and medical equipment wholesale...,"[{'city': 'Maribor', 'country': 'SLOVENIA', 'c...",4,3124533,...,NaN,"[-0.279010892, -0.0487989485, 0.415008128, -0....",medicine,health,medicine,"[0.013658804818987846, 0.0062546506524086, 0.0...",2023-01-05,NaN,NaN,NaN


# Pre-processing of article texts

## Remove schedules and reviews

In [37]:
articles = articles[~articles['categories'].str.contains('AD|SE|RU', case=False, na=False)]
articles = articles[~articles['keywords'].str.contains('REVIEW', case=False, na=False)]
articles_df = pd.DataFrame(articles[['headline', 'lede', 'text']])
articles_df.dropna(inplace=True)

In [38]:
articles_df

,headline,lede,text
3,Government and unions resume wage reform talks,"A month after their last meeting, the governme...",The negotiations come after the government rec...
4,Finance pessimistic about EU's green mobility ...,Looking at Europe's current policies regarding...,Europe's existing policies have not chased awa...
5,Weather: Mini cold snap,An incursion of cold air from the north will d...,Daytime highs will remain in the low teens on ...
7,Dnevnik says administrative units product of s...,The newspaper Dnevnik comments on Monday on th...,"On Friday, representatives of Slovenian munici..."
8,Govt rating keeps falling,The voter approval rating for the government k...,The government enjoys the support of 41.5% of ...
...,...,...,...
9278,Rights groups urge Koper port to implement out...,Labour rights groups urged port operator Luka ...,"Addressing the press in Koper, Damjan Volf of ..."
9279,Slovenian IoT provider GOAP acquired by Allterco,"Allterco, a Bulgarian developer of internet of...",The total value of the outstanding stake range...
9280,Slovenian co-operative allegedly victim of fraud,Slovenian police are investigating suspicion t...,The president of the Koroška Agricultural and ...
9281,Medical wholesaler Salus buys another rival,Pharmaceutical and medical equipment wholesale...,Farmadent was founded and owned by six municip...


## Remove HTML tags, split per section

In [39]:
import re
from html import unescape

def remove_html_tags(text):
    clean = re.compile('<.*?>', re.DOTALL)
    return re.sub(clean, '', text)

def split_into_sections(text):
    return re.split(r'\n\n+', text)

articles_df['text_processed'] = articles_df['text'].apply(unescape)
articles_df['text_processed'] = articles_df['text_processed'].apply(remove_html_tags)
articles_df['text_processed'] = articles_df['text_processed'].apply(split_into_sections)


# Embedding of headlines, lede and texts (around 20min of computations)

In [41]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def get_embeddings(text):
    return list(model.encode(text))


articles_df['headline_embeddings'] = articles_df['headline'].apply(get_embeddings)
articles_df['lede_embeddings'] = articles_df['lede'].apply(get_embeddings)
articles_df['text_mean_embeddings'] = articles_df['text_embeddings'].apply(lambda embeddings: [sum(x) / len(x) for x in zip(*embeddings)])

# Averaging the headline, lede and text embeddings, to get the article embedding

In [42]:
articles_df['average_embeddings'] = articles_df.apply(lambda row: [(a + b + c) / 3 for a, b, c in zip(row['text_mean_embeddings'], row['headline_embeddings'], row['lede_embeddings'])], axis=1)
articles_df['full_text'] = 'Headline: ' + articles_df['headline'] + '\nLede: ' + articles_df['lede'] + '\nText: ' + articles_df['text']

# Save result to .csv

In [43]:
articles_chroma = pd.DataFrame(articles_df[['full_text', 'average_embeddings']])
articles_chroma.columns = ['text', 'embeddings']
articles_chroma.to_csv('application/articles_chroma.csv', index=False)

# Adding most similar articles (in terms of cosine similarity) to IPTC categories to ChromaDB.

In [57]:
client = chromadb.HttpClient(host='localhost', port=8000)
collection = client.get_collection("database")

In [45]:
embeddings = list(articles_chroma['embeddings'])
texts = list(articles_chroma['text'])

query_result = collection.query(query_embeddings=embeddings, n_results=1)

In [53]:
articles = []
metadatas = []
embeddings2 = []

for i in range(len(query_result['ids'])):
    if 0.5 < query_result['distances'][i][0] < 0.9:
        articles.append(texts[i])
        metadatas.append(query_result['metadatas'][i][0])
        embeddings2.append(embeddings[i])

ids = [str(uuid.uuid4()) for i in range(len(articles))]
collection.add(
    ids =  ids,
    documents = articles,
    metadatas = metadatas,
    embeddings = embeddings2
    )